### Data Pre-Processing

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk

from nltk import tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
df_raw = pd.read_excel("amazon_review.xlsx")
df_raw.head(5)

,Review Model,Retailer,Review date,Review name,Review rating,Review title,Review Content,Verified Purchase or not,People_find_helpful,vine or not,URL,list price,rating count,overall rating
0,Canon Pixma TS6420a,Amazon,2023-08-10,Ernest Birkholz,5,Works great,🖨 was easy to install and works great.,Verified Purchase,NaN,NaN,https://www.amazon.com/product-reviews/B09TG52...,129.99,285,4.2
1,HP OfficeJet Pro 9015e,Amazon,2022-06-04,mattey,3,spunky mid size printer,🔆Slower print speed than what I’m used to (old...,Verified Purchase,NaN,NaN,https://www.amazon.com/product-reviews/B08QR6P...,289.99,285,4.2
2,Canon PIXMA MG3620,Amazon,2023-03-15,Maria D,4,𝙲𝚕𝚎𝚊𝚛 𝚙𝚛𝚒𝚗𝚝𝚜,𝙻𝚘𝚟𝚎 𝚒𝚝,Verified Purchase,NaN,NaN,https://www.amazon.com/Canon-MG3620-Wireless-P...,79.99,285,4.2
3,Epson - ET-3830,Amazon,2022-11-17,Ryan H,5,Shaq knows what he's talking about,"Yup, this printer is a slam dunk. :)What an up...",Verified Purchase,12.0,NaN,https://www.amazon.com/product-reviews/B096NBP...,399.99,285,4.2
4,HP ENVY 6055e,Amazon,2022-04-09,Sam,3,Not User Friendly,You would think something as simple as reconne...,Verified Purchase,NaN,NaN,https://www.amazon.com/product-reviews/B08XYRV...,129.99,285,4.2


In [10]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10209 entries, 0 to 10208
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Review Model              10209 non-null  object        
 1   Retailer                  10209 non-null  object        
 2   Review date               10209 non-null  datetime64[ns]
 3   Review name               10152 non-null  object        
 4   Review rating             10209 non-null  int64         
 5   Review title              10028 non-null  object        
 6   Review Content            10198 non-null  object        
 7   Verified Purchase or not  9425 non-null   object        
 8   People_find_helpful       3905 non-null   float64       
 9   vine or not               265 non-null    object        
 10  URL                       10209 non-null  object        
 11  list price                10209 non-null  float64       
 12  rating count      

#### Feature Extraction

In [11]:
# Number of words
df_raw['word_count'] = df_raw['Review Content'].apply(lambda x: len(str(x).split(" ")))
df_raw[['Review Content','word_count']].head()

,Review Content,word_count
0,🖨 was easy to install and works great.,8
1,🔆Slower print speed than what I’m used to (old...,79
2,𝙻𝚘𝚟𝚎 𝚒𝚝,2
3,"Yup, this printer is a slam dunk. :)What an up...",84
4,You would think something as simple as reconne...,160


In [12]:
# Number of characters
df_raw['char_count'] = df_raw['Review Content'].str.len() ## this also includes spaces
df_raw[['Review Content','char_count']].head()

,Review Content,char_count
0,🖨 was easy to install and works great.,38.0
1,🔆Slower print speed than what I’m used to (old...,460.0
2,𝙻𝚘𝚟𝚎 𝚒𝚝,7.0
3,"Yup, this printer is a slam dunk. :)What an up...",469.0
4,You would think something as simple as reconne...,861.0


In [17]:
# Average word length
def avg_word(sentence):
    if isinstance(sentence, str):  # Check if the input is a string
        words = sentence.split()
        return sum(len(word) for word in words) / len(words)
    else:
        return 0  # Return 0 for non-string values

df_raw['avg_word'] = df_raw['Review Content'].apply(lambda x: avg_word(x))
df_raw[['Review Content', 'avg_word']].head()

,Review Content,avg_word
0,🖨 was easy to install and works great.,3.875000
1,🔆Slower print speed than what I’m used to (old...,4.835443
2,𝙻𝚘𝚟𝚎 𝚒𝚝,3.000000
3,"Yup, this printer is a slam dunk. :)What an up...",4.595238
4,You would think something as simple as reconne...,4.387500


In [20]:
# Number of stop words
stop = stopwords.words('english')

def count_stopwords(sentence):
    if isinstance(sentence, str):  # Check if the input is a string
        words = sentence.split()
        return len([word for word in words if word in stop])
    else:
        return 0  # Return 0 for non-string values

df_raw['stopwords'] = df_raw['Review Content'].apply(lambda x: count_stopwords(x))
df_raw[['Review Content', 'stopwords']].head()

,Review Content,stopwords
0,🖨 was easy to install and works great.,3
1,🔆Slower print speed than what I’m used to (old...,19
2,𝙻𝚘𝚟𝚎 𝚒𝚝,0
3,"Yup, this printer is a slam dunk. :)What an up...",31
4,You would think something as simple as reconne...,71


#### Text Pre-Processing

In [ ]:
# Lower casing




In [ ]:
# Removing punctuation 



In [ ]:
# Removal of stop words




In [ ]:
# Removal of common words




In [ ]:
# Removal of rare words




In [ ]:
# Spelling correction 




In [ ]:
# Tokenisation 




In [ ]:
# Stemming 




In [ ]:
# Lemmatization 




In [ ]:
# Export to csv for use


